# SQL tasks
The bank will create a dedicated campaign and target these specific clients moving forward. In this step, help the bank find these clients.

## **1. How many clients does the bank have and are above the age of 50?**
1. How many clients does the bank have and are above the age of 50?

In [3]:
--Let's first figure out how many customers we have in total.
SELECT COUNT(customer_age)
FROM basic_client_info;

,count
0,10127


In [4]:
-- Okay, and now customers over 50.
SELECT COUNT(customer_age)
FROM basic_client_info
WHERE customer_age > 50;

,count
0,3078


## **2. What’s the distribution (in %) between male and female clients?**

In [5]:
--First try at building a query
    
WITH t1 AS(
SELECT *
FROM basic_client_info
WHERE gender = 'M'
),
t2 AS (
SELECT *
FROM basic_client_info
WHERE gender = 'F'
)
SELECT (COUNT(t1.gender))/(COUNT(t1.gender) + COUNT(t2.gender))::float*100 AS percent_male,
(COUNT(t2.gender))/(COUNT(t1.gender) + COUNT(t2.gender))::float*100 AS percent_female
FROM t1
FULL OUTER JOIN t2
ON t1.clientnum = t2.clientnum;

,percent_male,percent_female
0,47.091932,52.908068


In [6]:
--That query felt like I was taking extra unecessary steps. Let's figure out how to do it in one query.
    
SELECT ROUND(100*(SUM(CASE WHEN gender = 'M' THEN 1 ELSE 0 END)::NUMERIC)/COUNT(clientnum),1) AS percent_male,
ROUND(100*(SUM(CASE WHEN gender = 'F' THEN 1 ELSE 0 END)::NUMERIC)/COUNT(clientnum),1) AS percent_female
FROM basic_client_info;
    
--Much better!

,percent_male,percent_female
0,47.1,52.9


## **3. Let’s define a new variable called `age_group`:**
    - 10 < x ≤ 30
    - 30 < x ≤ 40
    - 40 < x ≤ 50
    - 50 < x ≤ 60
    - 60 <x ≤ 120
    
    Per each `age_group`, `marital_status` and `income_category`, find out the following values:
    
    a. Churn_rate (in %)
    
    b. Average `Total_Relationship_Count`
    
    c. Minimum value of `Total_Amt_Chng_Q4_Q1`
    
    d. Count of customers
    
    Make sure to order the data by the number of customers in a descending order***

In [2]:
WITH t1 AS(
SELECT clientnum,
CASE
WHEN customer_age <= 30 THEN '10 - 30'
WHEN customer_age <= 40 THEN '30 - 40'
WHEN customer_age <= 50 THEN '40 - 50'
WHEN customer_age <= 60 THEN '50 - 60'
ELSE '60 - 120'
END AS age_group
FROM basic_client_info
    ),
t2 AS(
SELECT clientnum, CASE WHEN attrition_flag = 'Attrited Customer' THEN 1 ELSE 0 END AS churn_flag
FROM bankchurners
)
SELECT age_group, 
round(AVG(churn_flag)*100.0,2) AS churn_rate, 
round(AVG(total_relationship_count),2) AS avg_tot_rel_cnt, 
MIN(total_amt_chng_q4_q1) AS min_total_amt_cng_q4_q1, 
COUNT(*) AS cust_count
FROM t1
JOIN t2 USING (clientnum)
JOIN enriched_churn_data USING (clientnum)
JOIN bankchurners USING (clientnum)
GROUP BY 1
ORDER BY 5 DESC;

,age_group,churn_rate,avg_tot_rel_cnt,min_total_amt_cng_q4_q1,cust_count
0,40 - 50,16.75,3.74,0.000,4652
1,50 - 60,16.76,3.77,0.000,2673
2,30 - 40,14.54,3.94,0.000,2132
3,60 - 120,14.32,4.12,0.278,405
4,10 - 30,12.08,3.99,0.299,265


## **4. Out of the male clients, who holds the “blue” card, how many (in %) hold the income category 40K - 60K?**

In [7]:
--Given that we calculated a percent question earlier, this one is a piece of cake! We just need to make sure we add a WHERE clause to filter only for male cients. 
    
SELECT ROUND(100*(SUM(CASE WHEN income_category = '$40K - $60K' THEN 1 ELSE 0 END)::NUMERIC)/COUNT(clientnum),1) AS 				percent_40k_to_60k
FROM basic_client_info
WHERE gender = 'M';

,percent_40k_to_60k
0,16.3


## **5. Without the usage of group by at all, find the 3rd and 4th highest client IDs (`CLIENTNUM`’s) of `Total_Amt_Chng_Q4_Q1`?**

In [9]:
--This one seems like a trick! 
    
SELECT e.clientnum, e.total_amt_chng_q4_q1
FROM enriched_churn_data e
ORDER BY 2 DESC
LIMIT 4;

,clientnum,total_amt_chng_q4_q1
0,710599683,3.397
1,710930508,3.355
2,713989233,2.675
3,713982108,2.594


In [10]:
--Now, I admit this was the easy way out. If they were asking for the 52nd highest client IDs the most efficient way of finding this would not be to use ORDER BY and scroll down. Instead, let's write a query that will show us only the two cliend IDs we're looking for (the 3rd and 4th highest)!
    
SELECT chng_filter.* 
FROM(
SELECT rank() OVER (ORDER BY e.total_amt_chng_q4_q1 DESC), e.clientnum, e.total_amt_chng_q4_q1
FROM enriched_churn_data e
)
chng_filter WHERE rank = 3;
    
--With this query you can specify the exact rank that you want to see the data for. This will be much more efficient if the request is for a rank of a higher number.

,rank,clientnum,total_amt_chng_q4_q1
0,3,713989233,2.675


In [11]:
--And the 4th!

SELECT chng_filter.* 
FROM(
SELECT rank() OVER (ORDER BY e.total_amt_chng_q4_q1 DESC), e.clientnum, e.total_amt_chng_q4_q1
FROM enriched_churn_data e
)
chng_filter WHERE rank = 4;

,rank,clientnum,total_amt_chng_q4_q1
0,4,713982108,2.594


## **6. We’re interested in knowing which client (CLIENTNUM) has the 2nd highest `Total_Trans_Amt`, Per each `Marital_Status`.**

In [8]:
	--If they were asking for just the 1rst highest we could use DISTINCT ON. However, they want the 2nd highest so I'll build a query using OVER and PARTITION BY. 
    
SELECT mar_filter.* 
FROM(
SELECT b.clientnum, rank() OVER (PARTITION BY b.marital_status ORDER BY e.total_trans_amt DESC), 
b.marital_status, 
e.total_trans_amt
FROM basic_client_info b
JOIN enriched_churn_data e
ON b.clientnum = e.clientnum
)
mar_filter WHERE rank = 2;

	--OVER tells the query how to window the output. PARTITION BY splits up the data into groups and the ORDER BY clause specifies how to order each of the groups. We utilize this to create a filter for the data. lastly, we use a WHERE clause to state that we only want to see the results for the 2nd rank. :) 

,clientnum,rank,marital_status,total_trans_amt
0,716894658,2,Divorced,16824
1,717642633,2,Married,17995
2,716004258,2,Single,17634
3,719848008,2,Unknown,16098
